In [3]:
!pip install --upgrade nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [62]:
import numpy as np
from scipy import sparse
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.sonority_sequencing import SyllableTokenizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from tqdm import tqdm
import warnings
SEED = 42

In [5]:
train_df = pd.read_csv("/kaggle/input/intromlda-2023-1-porn-detection/train.csv")
test_df = pd.read_csv("/kaggle/input/intromlda-2023-1-porn-detection/test.csv")

Очистим данные от непонятных символов и букв не русского и английского алфавита.

Удалим стопслова,а у url еще 'ru', 'com', 'www'. Заголовки обработаем стемингом, а url разобьем на слоги.

In [6]:
stemmer_eng = SnowballStemmer('english')
stemmer_rus = SnowballStemmer('russian')

In [7]:
def preprocessor_title(text):
    text = text.lower()
    text = re.sub(r"\s+" ," ", re.sub(r"[^a-zа-яё]", " ", text))
    tokens = []
    for word in text.split(' '):
        if (word in stopwords.words(['russian', 'english'])) or word=='':
            continue
        else: 
            tokens.append(word)
            
    for i in range (len(tokens)):
        tokens[i] = stemmer_eng.stem(tokens[i])
        tokens[i] = stemmer_rus.stem(tokens[i])
        
    return ' '.join(tokens)

In [8]:
syllabletoken = SyllableTokenizer()

def process_url(url):
    stop_ngram = ['ru', 'com', 'www', '', ' ']
    
    split = re.split('[^A-Za-z0-9]+', url)
    tokens = [token for token in split if token not in stop_ngram]
    
    ngrams = []
    for word in tokens:
        ngrams += syllabletoken.tokenize(word)
    
    return ' '.join(ngrams)

In [9]:
train_df[train_df.isna().any(axis=1)]

,ID,url,title,label
78497,78497,jpg-1.com,NaN,0


In [10]:
train_df.dropna(inplace=True)

In [12]:
train_df['title_tokens'] = train_df['title'].apply(lambda x: preprocessor_title(x))

In [13]:
train_df['url_token'] = train_df['url'].apply(lambda x: process_url(x))

In [15]:
train_df.to_parquet('/kaggle/working/train_with_tokens.parquet')

Попробуем LogisticRegression с разным векторайзерами

In [21]:
tfidf_vectorizer = TfidfVectorizer()
count_vectorizer = CountVectorizer()

In [14]:
df_train, df_test, y_train, y_test = train_test_split(train_df.drop('label', axis = 1), train_df.label, random_state=SEED)

In [25]:
X_train = df_train['url_token'].values + ' ' + df_train['title_tokens'].values
X_test = df_test['url_token'].values + ' ' + df_test['title_tokens'].values

In [26]:
tfidf_vectorizer.fit(X_train)
count_vectorizer.fit(X_train)

CountVectorizer()

In [32]:
vect_train_tfidf = tfidf_vectorizer.transform(X_train)
vect_train_count = count_vectorizer.transform(X_train)

In [45]:
model_log_rec = LogisticRegression( C=1000, n_jobs= 4)

In [47]:
model_log_rec.fit(vect_train_tfidf, y_train)
print('TfidfVectorizer train:', f1_score(y_train, model_log_rec.predict(vect_train_tfidf)))
model_log_rec.fit(vect_train_count, y_train)
print('CountVectorizer train:', f1_score(y_train, model_log_rec.predict(vect_train_count)))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TfidfVectorizer train: 0.9995609308266475
CountVectorizer train: 0.999600830273032


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
vect_test_tfidf = tfidf_vectorizer.transform(X_test)
vect_test_count = count_vectorizer.transform(X_test)

In [49]:
model_log_rec.fit(vect_train_tfidf, y_train)
print('TfidfVectorizer test:', f1_score(y_test, model_log_rec.predict(vect_test_tfidf)))
model_log_rec.fit(vect_train_count, y_train)
print('CountVectorizer test:', f1_score(y_test, model_log_rec.predict(vect_test_count)))

TfidfVectorizer test: 0.9742908871454435
CountVectorizer test: 0.9737095996140858


Попробуем LogisticRegressionCV с разным векторайзерами

In [51]:
preprocessor_cv = ColumnTransformer([
    ('url_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32), 'url_token'),
    ('title_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32), 'title_tokens'),   
])

    


model_cv = Pipeline([
    ('preprocessor', preprocessor_cv),
    ('linear_model', LogisticRegressionCV(class_weight = 'balanced', cv = 3))
])

In [53]:
model_cv.fit(df_train, y_train)
pred_train = model_cv.predict(df_train)
pred_test = model_cv.predict(df_test)
print('LogisticRegressionCV train TfidfVectorizer f1_score: ', f1_score(pred_train, y_train))
print('LogisticRegressionCV test TfidfVectorizer f1_score: ', f1_score(pred_test, y_test))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

LogisticRegressionCV train TfidfVectorizer f1_score:  0.9988436540531919
LogisticRegressionCV test TfidfVectorizer f1_score:  0.974697205899988


In [52]:
model_cv.fit(df_train, y_train)
pred_train = model_cv.predict(df_train)
pred_test = model_cv.predict(df_test)
print('LogisticRegressionCV train CountVectorizer f1_score: ', f1_score(pred_train, y_train))
print('LogisticRegressionCV test CountVectorizer f1_score: ', f1_score(pred_test, y_test))

LogisticRegressionCV train CountVectorizerf1_score:  0.9992820103709613
LogisticRegressionCV test CountVectorizer f1_score:  0.9760960960960962


Попробуем сделать ансамбль моделей, лучше, но на лидерборде все равно не так хорошо

In [22]:
preprocessor1 = ColumnTransformer([
    ('url_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32), 'url_token'),
    ('title_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32), 'title_tokens'),   
])
preprocessor2 = ColumnTransformer([
    ('url_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32), 'url_token'),
    ('title_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32), 'title_tokens'),   
])
preprocessor3 = ColumnTransformer([
    ('url_tokens', CountVectorizer(analyzer='word', dtype = np.float32), 'url_token'),
    ('title_tokens', CountVectorizer(analyzer='word', dtype = np.float32), 'title_tokens'),   
])
preprocessor4 = ColumnTransformer([
    ('url_tokens', CountVectorizer(analyzer='word', dtype = np.float32), 'url_token'),
    ('title_tokens', CountVectorizer(analyzer='word', dtype = np.float32), 'title_tokens'),   
])

    


model1 = Pipeline([
    ('preprocessor', preprocessor1),
    ('linear_model', LogisticRegressionCV(class_weight = 'balanced', cv = 3))
])
model2 = Pipeline([
    ('preprocessor', preprocessor2),
    ('linear_model', SGDClassifier(class_weight = 'balanced', loss='log_loss'))
])
model3 = Pipeline([
    ('preprocessor', preprocessor3),
    ('linear_model', LogisticRegression(class_weight = 'balanced'))
])
model4 = Pipeline([
    ('preprocessor', preprocessor4),
    ('linear_model', RandomForestClassifier(class_weight = 'balanced'))
])


ensamble = VotingClassifier(estimators=[('mod1', model1), ('mod2', model2),
                                     ('mod3', model3), ('mod4', model4)], voting = 'soft')

In [23]:
ensamble.fit(df_train, y_train)
pred_train = ensamble.predict(df_train)
pred_test = ensamble.predict(df_test)
print('Ensamble train f1_score: ', f1_score(pred_train, y_train))
print('Ensamble test f1_score: ', f1_score(pred_test, y_test))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Ensamble train f1_score:  0.9948215423836838
Ensamble test f1_score:  0.9752701080432173


Ансамбль тоже не показал себя сильно лучше. Скорее всего дела в фичах, надо пробовать делать новые. Найдем ключевые слова для заголовков и url

In [25]:
positive_texts_title = train_df.loc[train_df['label'] == 1, 'title_tokens'].tolist()
negative_texts_title = train_df.loc[train_df['label'] == 0, 'title_tokens'].tolist()

In [28]:
dictionary_pos = {}
for line in positive_texts_title:
    line = line.strip()
    for word in line.split():
        if word in dictionary_pos.keys():
            cur_value = dictionary_pos[word] + 1
            dictionary_pos[word] = cur_value
        else:
            dictionary_pos[word] = 1
sorted_dict_pos = [(k, dictionary_pos[k]) for k in sorted(dictionary_pos, key=dictionary_pos.get, reverse=True)]

In [33]:
dictionary_neg = {}
for line in negative_texts_title:
    line = line.strip()
    for word in line.split():
        if word in dictionary_neg.keys():
            cur_value = dictionary_neg[word] + 1
            dictionary_neg[word] = cur_value
        else:
            dictionary_neg[word] = 1
sorted_dict_neg = [(k, dictionary_neg[k]) for k in sorted(dictionary_neg, key=dictionary_neg.get, reverse=True)]

In [34]:
keys_only_in_pos = [key for key, num in sorted_dict_pos if key not in sorted_dict_neg]

In [37]:
keys_only_in_pos[:10]

['порн',
 'porn',
 'vide',
 'виде',
 'sex',
 'онлайн',
 'hd',
 'секс',
 'xxx',
 'смотрет']

удалим элементы ссылок

In [36]:
keys_only_in_pos.remove('com')
keys_only_in_pos.remove('www')
keys_only_in_pos.remove('ru')

In [42]:
positive_texts_url = train_df.loc[train_df['label'] == 1, 'url_token'].tolist()
negative_texts_url = train_df.loc[train_df['label'] == 0, 'url_token'].tolist()

dictionary_pos_url = {}
for line in positive_texts_url:
    line = line.strip()
    for word in line.split():
        if word in dictionary_pos_url.keys():
            cur_value = dictionary_pos_url[word] + 1
            dictionary_pos_url[word] = cur_value
        else:
            dictionary_pos_url[word] = 1
sorted_dict_pos_url = [(k, dictionary_pos_url[k]) for k in sorted(dictionary_pos_url, key=dictionary_pos_url.get, reverse=True)]

dictionary_neg_url = {}
for line in negative_texts_url:
    line = line.strip()
    for word in line.split():
        if word in dictionary_neg_url.keys():
            cur_value = dictionary_neg_url[word] + 1
            dictionary_neg_url[word] = cur_value
        else:
            dictionary_neg_url[word] = 1
sorted_dict_neg_url = [(k, dictionary_neg_url[k]) for k in sorted(dictionary_neg_url, key=dictionary_neg_url.get, reverse=True)]

keys_only_in_pos_url = [key for key, num in sorted_dict_pos_url if key not in sorted_dict_neg_url]

In [43]:
keys_only_in_pos_url[:10]

['por', 'no', 'net', 'porn', 'bi', 'org', 'tu', 'me', 'mo', 'be']

отдельно выделим русские и английские, русские будем использовать больше

In [47]:
russian_word_pattern = re.compile(r'^[а-я]+$')
russian_words = [key for key in keys_only_in_pos if russian_word_pattern.match(key)]
english_word_pattern = re.compile(r'^[a-z]+$')
english_words = [key for key in keys_only_in_pos if english_word_pattern.match(key)]

создадим функцию для создания матрицы фич

In [48]:
def features(df):    
    title = df["title"].values
    url = df["url"].values
    result = []
    for i, val in enumerate(title):
        result.append([])
        for j in russian_words[:800]:
            result[-1].append(int(j in val.lower()))
        for j in english_words[:200]:
            result[-1].append(int(j in val.lower()))
        for j in keys_only_in_pos_url[:200]:
            result[-1].append(int(j in url[i].lower()))
    return result
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegressionCV 

In [ ]:
# df_train, df_test, y_train, y_test = train_test_split(train_df.drop('label', axis = 1), train_df.label, random_state=SEED)

In [45]:
X_train = df_train["title"].values + ' ' + df_train["url"].values
X_test = df_test["title"].values+ ' '+df_test["url"].values

In [50]:
vectorizer = CountVectorizer()
X_train_vectorized = sparse.hstack((vectorizer.fit_transform(X_train), sparse.csr_matrix(np.asarray(features(df_train)))))

model =  LinearSVC(class_weight='balanced')
model.fit(
    X_train_vectorized,
    y_train
)
y_pred = model.predict(
    X_train_vectorized
)
print(f1_score(y_train, y_pred))

0.999680817108203


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [52]:
X_test_vectorized = sparse.hstack((vectorizer.transform(X_test), sparse.csr_matrix(np.asarray(features(df_test)))))
y_pred_test = model.predict(
    X_test_vectorized
)
print(f1_score(y_test, y_pred_test))

0.9823083403538332


как вижно эта модель показывает значительных прирост, если поиграться с количеством ключевых слов. Но слишком много использовать не получается так как сильно увеличивается время

In [53]:
X_train = train_df["title"].values + ' ' + train_df["url"].values
X_test = test_df["title"].values+ ' '+test_df["url"].values
y_train = train_df["label"].astype(int).values

In [54]:
vectorizer = CountVectorizer()
X_train_vectorized = sparse.hstack((vectorizer.fit_transform(X_train), sparse.csr_matrix(np.asarray(features(train_df)))))

model =  LinearSVC(class_weight='balanced')
model.fit(
    X_train_vectorized,
    y_train
)
y_pred = model.predict(
    X_train_vectorized
)
print(f1_score(y_train, y_pred))

0.999611255644269


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Это оказался самый лучший подход, были различные эксперименты с количеством ключевых слов russian_words[:800], так как при большом значение матрица создается долго, но при этом увеличивается точность. Также были различные эксперименты с моделями (LinearSVC, RandomForestClassifier, SGDClassifier, LogisticRegression, LogisticRegressionCV), но LinearSVC, показала лучший результат на лидерборде с параметрами по умолчанию.

Также я пробовал обучить ансамбль на этих данных, но он слишком долго учится и результаты почему-то лучше не получились.

Также проанализиров данные, можно сделать предположения что дополнительно можно проверять не был ли этот сайт замечен с запрещенным контентом в трейне, а также нет ли в заголовке слов, которые, очевидно, говорят, о запрещенном контентом.

In [55]:
X_test_vectorized = sparse.hstack((vectorizer.transform(X_test), sparse.csr_matrix(np.asarray(features(test_df)))))

test_df["label"] = model.predict(X_test_vectorized).astype(int)

In [56]:
test_df.head()

,ID,url,title,label
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...,0
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ...",0
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12,0
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ...",1
4,135313,2gis.ru,67,0


Создадим список с запрещенными сайтами, для этого возьмем те сайты, которые были замечены с запрещенным контентов в трейне более 2 раз. Пробовал более 1, но это ухудшало результат на лидерборде.

In [58]:
prohibited_sites = []
for url in train_df.groupby('url'):
    if ( url[1].label.count() > 2 ):
        if ( all(url[1].label) ):
            prohibited_sites.append(url[0])
print(len(prohibited_sites))

951


Также дополнительно будем проверять нет ли в заголовках или url, ключевых самых популярных запрещенных слов

In [59]:
test_df['label'] = test_df.apply(lambda x:  int(x['label'] or 
                                            ('porn' in x['url']) or
                                            ('xxx' in x['url']) or
                                            ('xvideo' in x['url']) or (' порно ' in x['title'].lower()) or ('porn' in x['title'].lower()) or (x['url'] in prohibited_sites)), axis=1)

In [60]:
test_df[["ID", "label"]].to_csv("/kaggle/working/solution.csv", index=False)

По итогу удалось обучить относительно хорошую модель не применяя сложных преобразований, энкодеров и других сложных моделей трубующих gpu.